### Package Python

In [1]:
import pandas as pd
import numpy as np
import xarray as xr
import os as os

### Reading of bases

In [11]:
_path = os.getcwd()
_path

'c:\\git\\Gol_Cases'

In [12]:
_path = os.path.join(_path, 'data', 'Case Analytics-DS.xlsx')
_path

'c:\\git\\Gol_Cases\\data\\Case Analytics-DS.xlsx'

In [14]:
_df = pd.read_excel(io=_path, sheet_name='Dados')
_df.head(2)

,Data Venda,Canal de Venda,Local de Venda,PAX,Monetário Vendido,RPK
0,2016-11-01,Telégrafo,Mindscape,30,2188.109487,3476
1,2016-11-01,TeleVenda,Arena,52,3302.375951,8197


In [15]:
_df.dtypes

Data Venda           datetime64[ns]
Canal de Venda               object
Local de Venda               object
PAX                           int64
Monetário Vendido           float64
RPK                           int64
dtype: object